# Data Preparation and Libraries
Import required libraries (sklearn, nltk) and set up the basic NLP components for text processing.

In [ ]:
# Import required libraries
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Set up stop words
stop_words = set(stopwords.words('english'))

# training phrases for the sports event assistant chatbot
training_phrases = {
    "jersey-purchase": ' '.join([
        "Do I need to bring a jersey?",
        "Should I wear a jersey?",
        "Is a jersey required?",
        "Do I need a team jersey?",
        "Should I buy a jersey?"
    ]),
    "food-order": ' '.join([
        "Can I order food?",
        "Is food available?",
        "Can I get food delivered?",
        "Do you have food options?",
        "Can I pick up food?"
    ]),
    "beverage-order": ' '.join([
        "Can I order beverages?",
        "Is beer available?",
        "Can I get drinks delivered?",
        "Do you have beverage options?",
        "Can I pick up drinks?"
    ]),
    "seat-navigation": ' '.join([
        "Help me find my seat",
        "Where is my seat?",
        "Can you guide me to my seat?",
        "I need help with seat navigation",
        "How do I find my seat?"
    ])
}

# Collect all training documents as user utterances
training_documents = list(training_phrases.values())
labels = list(training_phrases.keys())

# Tokenize the user utterances
word_tokens = [word_tokenize(sent) for sent in training_documents]

# Filter out stop words
filtered_sentences = [[w for w in tokens if not w in stop_words] for tokens in word_tokens]

# Create Bag of Words Model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(training_documents)
feature_names = vectorizer.get_feature_names_out()

# Create a Classifier
classifier = MultinomialNB()
classifier.fit(X, labels)

# Training Phrases and Intents
Create training data with intents:
- jersey-inquiry (size, availability, team options)
- food-delivery (pre-order, delivery to seat)
- beverage-service (beer packages, drink options)
- seat-navigation (directions, gate info)
- parking-info (lot locations, pricing)
- group-booking (group size limits, package deals)

In [ ]:
# Add new intents to the training phrases
training_phrases.update({
    "parking-info": ' '.join([
        "Where can I park?",
        "Is there parking available?",
        "What are the parking options?",
        "How much is parking?",
        "Where is the parking lot?"
    ]),
    "group-booking": ' '.join([
        "Can I book for a group?",
        "What are the group booking options?",
        "How many people can I book for?",
        "Are there group discounts?",
        "Can I get a package deal for a group?"
    ])
})

# Update training documents and labels
training_documents = list(training_phrases.values())
labels = list(training_phrases.keys())

# Tokenize the user utterances
word_tokens = [word_tokenize(sent) for sent in training_documents]

# Filter out stop words
filtered_sentences = [[w for w in tokens if not w in stop_words] for tokens in word_tokens]

# Create Bag of Words Model
X = vectorizer.fit_transform(training_documents)
feature_names = vectorizer.get_feature_names_out()

# Train the classifier with updated data
classifier.fit(X, labels)

# Text Processing
Implement text preprocessing pipeline:
- Tokenization
- Stop word removal
- Bag of words creation
- Handle common sports venue terminology

In [ ]:
# Tokenize the user utterances
word_tokens = [word_tokenize(sent) for sent in training_documents]

# Filter out stop words
filtered_sentences = [[w for w in tokens if not w in stop_words] for tokens in word_tokens]

# Create Bag of Words Model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(training_documents)
feature_names = vectorizer.get_feature_names_out()

# Handle common sports venue terminology
sports_venue_terms = {
    "jersey": ["shirt", "uniform"],
    "food": ["snacks", "meals"],
    "beverages": ["drinks", "beer"],
    "seat": ["place", "spot"],
    "parking": ["lot", "garage"],
    "group": ["team", "party"]
}

def replace_synonyms(text, synonyms_dict):
    for key, synonyms in synonyms_dict.items():
        for synonym in synonyms:
            text = text.replace(synonym, key)
    return text

# Apply synonym replacement to training documents
training_documents = [replace_synonyms(doc, sports_venue_terms) for doc in training_documents]

# Recreate Bag of Words Model with updated training documents
X = vectorizer.fit_transform(training_documents)
feature_names = vectorizer.get_feature_names_out()

# Train the classifier with updated data
classifier.fit(X, labels)

# Model Creation
Build and train classifier:
- Use MultinomialNB
- Create feature vectors
- Train on intent data
- Implement prediction function

In [ ]:
# Implement prediction function
def predict(raw_queries):
    # Replace synonyms in the raw queries
    processed_queries = [replace_synonyms(query, sports_venue_terms) for query in raw_queries]
    
    # Transform the queries using the vectorizer
    queries_transformed = vectorizer.transform(processed_queries)
    
    # Run prediction on the transformed queries
    return classifier.predict(queries_transformed)

# Example usage of the prediction function
predicted = predict([
    "Can I get a beer?",
    "Where is the parking garage?",
    "Do I need to wear a uniform?",
    "Can you guide me to my spot?",
    "Can I book for a team?"
])

predicted

# Response Generation
Create response templates for each intent:
- Jersey sizing and availability responses
- Food/beverage menu and ordering info
- Navigation instructions with landmarks
- Parking and entry guidance

In [ ]:
# Response Generation

# Define response templates for each intent
responses = {
    "jersey-purchase": [
        "You don't need to bring a jersey, but it's always great to show team spirit!",
        "Wearing a jersey is optional, but it can enhance your experience.",
        "A jersey is not required, but many fans love to wear them.",
        "You can buy a jersey at the venue or online before the event.",
        "It's not mandatory to have a team jersey, but it adds to the fun!"
    ],
    "food-order": [
        "Yes, you can order food at the venue.",
        "Food is available for purchase at various stands.",
        "You can get food delivered to your seat.",
        "We have a variety of food options available.",
        "You can pick up food from the designated areas."
    ],
    "beverage-order": [
        "Yes, you can order beverages including beer.",
        "Beer and other drinks are available at the venue.",
        "You can get drinks delivered to your seat.",
        "We have a variety of beverage options available.",
        "You can pick up drinks from the designated areas."
    ],
    "seat-navigation": [
        "I can help you find your seat. Please follow the signs.",
        "Your seat is located in section A. Follow the signs for section A.",
        "You can find your seat by looking at the row and seat number on your ticket.",
        "Ask an usher for help if you have trouble finding your seat.",
        "Use the venue map to navigate to your seat."
    ],
    "parking-info": [
        "You can park in the main parking lot.",
        "Parking is available at the venue for a fee.",
        "There are several parking options around the venue.",
        "Parking costs $20 for the event.",
        "The parking lot is located next to the main entrance."
    ],
    "group-booking": [
        "Yes, you can book tickets for a group.",
        "We offer group booking options for large parties.",
        "You can book tickets for up to 20 people.",
        "Group discounts are available for parties of 10 or more.",
        "We offer package deals for group bookings."
    ]
}

# Function to generate a response based on the predicted intent
def generate_response(predicted_intent):
    import random
    return random.choice(responses[predicted_intent])

# Example usage of the response generation function
predicted_intent = predict(["Can I get a beer?"])[0]
response = generate_response(predicted_intent)
response

# Interactive Testing
Test chatbot with sample queries:
- 'Where can I buy team jerseys?'
- 'How do I order beer for my group?'
- 'Directions to Section 124?'
- Evaluate accuracy and response relevance

In [ ]:
# Interactive Testing

# Define sample queries for testing
sample_queries = [
    "Where can I buy team jerseys?",
    "How do I order beer for my group?",
    "Directions to Section 124?"
]

# Predict intents for the sample queries
predicted_intents = predict(sample_queries)

# Generate responses for the predicted intents
responses = [generate_response(intent) for intent in predicted_intents]

# Display the sample queries, predicted intents, and generated responses
for query, intent, response in zip(sample_queries, predicted_intents, responses):
    print(f"Query: {query}")
    print(f"Predicted Intent: {intent}")
    print(f"Response: {response}")
    print()